## Importando bibliotecas 

In [1]:
#Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import math

#from io import StringIO, BytesIO
#from urllib.request import urlopen
from scipy.io.arff import loadarff

import config

In [2]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [3]:
import tensorflow as tf
from tensorflow import keras
from keras import layers , models
from keras.layers import Flatten, Dense, Dropout, Input
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential, load_model


## A fazer

Fazer um código que funcione para todos os datasets

## Duvidas


1.   Não é necessário normalizar os dados antes?
2.   É preciso fazer algum tipo de vizualizaçao de dados ou de resultados




## Funções

### Modelos

In [13]:
#DecisionTree
def model_dtree(x_train,y_train,x_test,y_test,booster,eta,max_depth,colsample_bytree,subsample,min_child_weight,reg_lambda,num_boost,my_num):
    bst = DecisionTreeRegressor().fit(x_train, y_train)
    predict = bst.predict(x_test)
    return predict

#RandomForestRegressor
def model_r(x_train,y_train,x_test,y_test,my_num):
    #booster,eta,max_depth,colsample_bytree,subsample,min_child_weight,reg_lambda,num_boost):
    bst = RandomForestRegressor(n_estimators = my_num,n_jobs = -1).fit(x_train,y_train)
    predict = bst.predict(x_test)
    return predict

#SVR
def model_svr(x_train,y_train,x_test,y_test,ker,gam,c,epsi,shrink,verb,max_it):
    bst = SVR(kernel=ker, gamma=gam, C=c, epsilon=epsi, shrinking=shrink, verbose=verb, max_iter=max_it).fit(x_train,y_train)
    predict = bst.predict(x_test)
    return predict

def model_multi_dtree(x_train, y_train, x_test, y_test):
    model = DecisionTreeRegressor()
    bst = MultiOutputRegressor(model).fit(x_train, y_train)
    predict = bst.predict(x_test)
    return predict

def model_ann(x_train, y_train, x_test, y_test):
    model = Sequential()
    model.add(Input(shape = (data.shape[1])))
    model.add(Dense(64, input_dim=4, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(y_train.shape[1], activation='sigmoid'))

    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=10, batch_size=32, verbose=0, validation_split=0.2)
    predict = model.predict(x_test)
    
    return predict

### Cálculo de erro 

In [5]:
def computer_RRMSE_list(real_test,result_p,real_train_mean):
    """RRMESE: Relative Root Mean Square Error
        input: real_test: real test data
              result_p: predicted data
              real_train_mean: mean of real train data
        output: RRMSE list """

    _list = []
    for i in range(result_p.shape[1]):
        fenzi = 0
        fenmu = 0
        for j in range(result_p.shape[0]):
            fenzi += (result_p[j,i] - real_test[j,i])**2
            fenmu += (real_train_mean[i] - real_test[j,i])**2
        _list.append(math.sqrt(fenzi/fenmu))    
    return _list

## Implementação

In [6]:
path = "../TCC/mtr-datasets/"

#text_file = input("dataset: ")
text_file = 'atp1d'
data, meta = loadarff(path + text_file + ".arff")
data = pd.DataFrame(data)
data.reset_index(inplace=True)
data.replace('?', np.nan, inplace=True)
data.replace('     ?', np.nan, inplace=True)
data = data.applymap(float)

print(data.shape)

In [7]:
if config.all_config[text_file].get('sample_random') == True:
    data = data.sample(frac=1,random_state = config.all_config[text_file].get('sample_random_num')).reset_index(drop=True)
data = data.fillna(pd.Series(np.nanmean(data,axis=0),index=data.columns))
label = data.iloc[:,-config.all_config[text_file].get('targets_num'):].values
data = data.iloc[:,:-config.all_config[text_file].get('targets_num')].values

In [10]:
# Teste Multi-dtree
loss_list = [] 
kf = KFold(n_splits=config.all_config[text_file].get('paper_kFold'), shuffle=config.all_config[text_file].get('kfold_random'))
for train_index , test_index in kf.split(data):
    x_train, x_test = data[train_index], data[test_index]
    y_train, y_test = label[train_index], label[test_index]
    result_p_train = []
    result_p_test = []
    r1 = model_multi_dtree(x_train, y_train, x_test, y_test)
    loss_list = computer_RRMSE_list(y_test,r1,np.mean(y_train,axis=0))
    
print("RRMSE: ", np.mean(loss_list))

RRMSE:  0.43056804336541504


In [14]:
loss_list_RRMSE = []
kf = KFold(n_splits=config.all_config[text_file].get('paper_kFold'),shuffle=config.all_config[text_file].get('kfold_random'),random_state=config.all_config[text_file].get('kfold_random_num'))
for train_index, test_index in kf.split(data):
    x_train, x_test = data[train_index], data[test_index]
    y_train, y_test = label[train_index], label[test_index]
    result_p_train = []
    result_p_test = []
    r1_list = []
    r1 = None 
    for i in range(label.shape[1]):
        if text_file in ['sf1', 'sf2']:
            r1 = model_svr(x_train,y_train[:,i],x_test,y_test[:,i],
                            config.all_config[text_file].get('svr1_kernel'),
                            config.all_config[text_file].get('svr1_gamma'),
                            config.all_config[text_file].get('svr1_C'),
                            config.all_config[text_file].get('svr1_epsilon'),
                            config.all_config[text_file].get('svr1_shrinking'),
                            config.all_config[text_file].get('svr1_verbose'),
                            config.all_config[text_file].get('svr1_max_iter'))
                        #    'rbf',0.1,100,0.8,True,True,False,10000)
        else:
            r1 = model_r(x_train,y_train[:,i],x_test,y_test[:,i], my_num=100)
        r1_list.append(r1)
    result_p_train.append(np.concatenate(r1_list,axis=0))
    predicted_test = None 
    if text_file in ['sf1', 'sf2']:
        predicted_test = model_svr(x_train, y_train[:,i], x_test, y_test[:,i],
                                   config.all_config[text_file].get('svr1_kernel'),
                                   config.all_config[text_file].get('svr1_gamma'),
                                   config.all_config[text_file].get('svr1_C'),
                                   config.all_config[text_file].get('svr1_epsilon'),
                                   config.all_config[text_file].get('svr1_shrinking'),
                                   config.all_config[text_file].get('svr1_verbose'),
                                   config.all_config[text_file].get('svr1_max_iter'))
    else:
        predicted_test = model_r(x_train, y_train[:,i], x_test, y_test[:,i], my_num=100)
    result_p_test.append(predicted_test)
result_p_train = pd.DataFrame(result_p_train).T.values
result_p_test = pd.DataFrame(result_p_test).T.values
real_train_mean = y_train.mean(axis=0)
loss_list_RRMSE.append(computer_RRMSE_list(y_test,result_p_test,real_train_mean))
loss_list_RRMSE_np = np.array(loss_list_RRMSE)

print('RRMSE: ',loss_list_RRMSE_np.mean(axis=0))



RRMSE:  [1.94205537]
1.942055367322777


In [9]:
# Teste ANN
# Ainda não está pronto
loss_list = [] 
kf = KFold(n_splits=config.all_config[text_file].get('paper_kFold'), shuffle=config.all_config[text_file].get('kfold_random'))
for train_index , test_index in kf.split(data):
    # print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = data[train_index], data[test_index]
    y_train, y_test = label[train_index], label[test_index]
  
    result_p_train = []
    result_p_test = []
    r1_list = []
    
    for i in range(label.shape[1]):
        r1 = model_ann(x_train, y_train, x_test, y_test)
        r1_list.append(r1)
    result_p_train.append(np.concatenate(r1_list,axis=0))

    loss_list = computer_RRMSE_list(y_test,result_p_train,np.mean(y_train,axis=0))
    
print("RRMSE: ", np.mean(loss_list))

2023-08-07 14:47:55.817479: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2/2 [==============================] - 0s 2ms/step


AttributeError: 'list' object has no attribute 'shape'

### Carregando Datasets (desatualizado)

In [ ]:
# path = '/content/drive/MyDrive/TCC/mtr-datasets/'

# andro = path + 'andro.arff'
# atp1d = path + 'atp1d.arff'
# atp7d = path + 'atp7d.arff'
# edm = path + 'edm.arff'
# enb = path + 'enb.arff'
# jura = path + 'jura.arff'
# oes10 = path + 'oes10.arff'
# oes97 = path + 'oes97.arff'
# osales = path + 'osales.arff'
# rf1 = path + 'rf1.arff'
# rf2 = path + 'rf2.arff'
# scm1d = path + 'scm1d.arff'
# scm20d = path + 'scm20d.arff'
# scpf = path + 'scpf.arff'
# sf1 = path + 'sf1.arff'
# sf2 = path + 'sf2.arff'
# slump = path + 'slump.arff'
# wq = path + 'wq.arff'

In [ ]:
# data_andro, meta_andro = loadarff(andro)
# df_andro = pd.DataFrame(data_andro)

# data_atp1d, meta_atp1d = loadarff(atp1d)
# df_atp1d = pd.DataFrame(data_atp1d)

# data_atp7d, meta_atp7d = loadarff(atp7d)
# df_atp7d = pd.DataFrame(data_atp7d)

# data_edm, meta_edm = loadarff(edm)
# df_edm = pd.DataFrame(data_edm)

# data_enb, meta_enb = loadarff(enb)
# df_enb = pd.DataFrame(data_enb)

# data_jura, meta_jura = loadarff(jura)
# df_jura = pd.DataFrame(data_jura)

# data_jura, meta_jura = loadarff(jura)
# df_jura = pd.DataFrame(data_jura)

In [ ]:
# data_oes10, meta_oes10 = loadarff(oes10)
# df_oes10 = pd.DataFrame(data_oes10)

# data_oes97, meta_oes97 = loadarff(oes97)
# df_oes97 = pd.DataFrame(data_oes97)

# data_osales, meta_osales = loadarff(osales)
# df_osales = pd.DataFrame(data_osales)

# data_rf1, meta_rf1 = loadarff(rf1)
# df_rf1 = pd.DataFrame(data_rf1)

# data_rf2, meta_rf2 = loadarff(rf2)
# df_rf2 = pd.DataFrame(data_rf2)

# data_scm1d, meta_scm1d = loadarff(scm1d)
# df_scm1d = pd.DataFrame(data_scm1d)

# data_scm20d, meta_scm20d = loadarff(scm20d)
# df_scm20d = pd.DataFrame(data_scm20d)

In [ ]:
# data_scpf, meta_scpf = loadarff(scpf)
# df_scpf = pd.DataFrame(data_scpf)

# data_sf1, meta_sf1 = loadarff(sf1)
# df_sf1 = pd.DataFrame(data_sf1)

# data_sf2, meta_sf2 = loadarff(sf2)
# df_sf2 = pd.DataFrame(data_sf2)

# data_slump, meta_slump = loadarff(slump)
# df_slump = pd.DataFrame(data_slump)

# data_wq, meta_wq = loadarff(wq)
# df_wq = pd.DataFrame(data_wq)